In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import syllables
import re
import stanza
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [2]:
pos = pd.read_csv('../Data/POS_Columns.csv',)

In [3]:
df = pd.read_csv('../Data/train.csv', encoding = 'utf-8', lineterminator="\n")

In [4]:
nlp = stanza.Pipeline('en', logging_level='WARN', processors='tokenize,pos,constituency, sentiment')

In [5]:
text = df['excerpt']

In [6]:
def convert_string(excerpt):
    string = ''

    for i in excerpt:
        i = i.replace("\n", "")
        string += i
    return string

In [7]:
test = convert_string(text[1])

In [8]:
doc = nlp(test)

In [9]:
for sent in doc.sentences:
    tree = sent.constituency
    print(sent.sentiment)
#     print(tree.children)
#     for child in tree.children:
#         print(child.children)
    

1
1
2
2
1
2
1
0
2
2
2
0
